# 股票数据缓存脚本使用示例

本示例演示如何在聚宽Jupyter Notebook环境中使用股票数据缓存脚本。

## 1. 导入必要的模块

首先导入所需的模块和配置。

In [ ]:
# 导入股票数据缓存器
from stock_cache import StockCache

# 查看默认配置
from config import pool_stocks, jq_params, cache_config, storage_config

print("股票池配置:")
for category, stocks in pool_stocks.items():
    print(f"  {category}: {stocks}")

print("\n聚宽API参数:")
for key, value in jq_params.items():
    print(f"  {key}: {value}")

## 2. 创建缓存器实例

创建股票数据缓存器实例，可以传入自定义配置。

In [ ]:
# 创建缓存器（使用默认配置）
cache = StockCache()

# 或者使用自定义配置
custom_config = {
    'pool_stocks': {
        '测试板块': ['000001', '600000']
    },
    'jq_params': {
        'start_date': '2024-01-01',
        'end_date': '2024-12-31',
        'frequency': 'daily'
    }
}

# cache = StockCache(custom_config)

## 3. 查看可用分类和股票

查看当前配置中的股票分类和对应的股票列表。

In [ ]:
# 查看所有分类
categories = cache.list_categories()
print(f"可用分类: {categories}")

# 查看特定分类的股票
for category in categories:
    stocks = cache.get_stocks_by_category(category)
    print(f"\n{category} 板块:")
    for stock in stocks:
        print(f"  {stock}")

## 4. 下载股票数据

下载所有股票数据或指定分类的股票数据。

In [ ]:
# 下载所有股票数据
print("开始下载所有股票数据...")
results = cache.download_all_stocks()

# 查看下载结果
success_count = sum(results.values())
print(f"\n下载完成: {success_count}/{len(results)} 成功")

# 或者只下载特定分类的数据
# results = cache.update_cache(['核心股'])

## 5. 获取和使用股票数据

获取已缓存的股票数据进行分析。

In [ ]:
# 获取单只股票数据
stock_code = '601888.XSHG'  # 中国中免
data = cache.get_stock_data(stock_code)

if data is not None:
    print(f"股票 {stock_code} 数据信息:")
    print(f"  数据期间: {data.index[0].strftime('%Y-%m-%d')} 到 {data.index[-1].strftime('%Y-%m-%d')}")
    print(f"  数据行数: {len(data)}")
    print(f"  数据字段: {list(data.columns)}")
    
    # 显示数据前几行
    print("\n数据前5行:")
    print(data.head())
    
    # 基本统计分析
    print("\n基本统计信息:")
    print(data.describe())
else:
    print(f"无法获取股票 {stock_code} 的数据")

## 6. 数据可视化示例

使用获取的数据进行简单的可视化分析。

In [ ]:
# 数据可视化
import matplotlib.pyplot as plt
import seaborn as sns

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 绘制价格走势图
if data is not None and not data.empty:
    plt.figure(figsize=(12, 8))
    
    # 子图1: 价格走势
    plt.subplot(2, 1, 1)
    plt.plot(data.index, data['close'], label='收盘价', linewidth=1)
    plt.title(f'{stock_code} 价格走势')
    plt.ylabel('价格(元)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # 子图2: 成交量
    plt.subplot(2, 1, 2)
    plt.bar(data.index, data['volume'], alpha=0.7, color='orange')
    plt.title('成交量')
    plt.ylabel('成交量')
    plt.xlabel('日期')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 计算收益率
    returns = data['close'].pct_change().dropna()
    
    # 收益率分布
    plt.figure(figsize=(10, 6))
    plt.hist(returns, bins=50, alpha=0.7, edgecolor='black')
    plt.title(f'{stock_code} 收益率分布')
    plt.xlabel('日收益率')
    plt.ylabel('频次')
    plt.grid(True, alpha=0.3)
    plt.show()

## 7. 批量处理多个股票

批量处理多个股票的数据。

In [ ]:
# 批量获取多个股票的数据
target_stocks = ['601888.XSHG', '002594.XSHE']  # 中国中免、比亚迪

all_data = {}
for stock in target_stocks:
    stock_data = cache.get_stock_data(stock)
    if stock_data is not None:
        all_data[stock] = stock_data
        print(f"成功获取 {stock} 数据，形状: {stock_data.shape}")
    else:
        print(f"获取 {stock} 数据失败")

# 对比分析多个股票
if len(all_data) >= 2:
    plt.figure(figsize=(12, 6))
    
    for stock, data in all_data.items():
        # 标准化价格（以第一个交易日为基准）
        normalized_price = data['close'] / data['close'].iloc[0]
        plt.plot(normalized_price.index, normalized_price.values, label=stock, linewidth=1.5)
    
    plt.title('多股票价格走势对比（标准化）')
    plt.ylabel('标准化价格')
    plt.xlabel('日期')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

## 8. 增量更新数据

当需要更新到最新数据时，可以使用增量更新功能。

In [ ]:
# 增量更新数据（只下载新增的数据）
print("开始增量更新数据...")
update_results = cache.update_cache()

# 或者强制全量更新
# update_results = cache.download_all_stocks(force_full=True)

## 9. 自定义配置示例

展示如何自定义配置以满足特定需求。

In [ ]:
# 自定义配置示例
custom_config = {
    'pool_stocks': {
        '科技股': ['000977', '300750', '688981'],
        '金融股': ['601318', '600036', '000001']
    },
    'jq_params': {
        'start_date': '2023-01-01',
        'end_date': '2024-12-31',
        'frequency': 'daily',
        'fields': ['open', 'high', 'low', 'close', 'volume', 'money'],
        'fq': 'pre'
    },
    'cache': {
        'update_strategy': 'incremental',
        'max_retry': 5
    },
    'storage': {
        'base_path': './my_stock_data',
        'format': 'parquet'
    }
}

# 使用自定义配置创建缓存器
custom_cache = StockCache(custom_config)

# 下载数据
custom_cache.download_all_stocks()

## 总结

本脚本提供了以下功能：

1. **智能缓存管理**：自动检测缓存状态，支持增量下载
2. **多股票批量处理**：支持按板块分类管理股票
3. **聚宽API集成**：直接调用聚宽API获取高质量数据
4. **灵活配置**：支持自定义股票池和参数配置
5. **高效存储**：使用Parquet格式存储，节省空间

使用建议：
- 首次使用时建议运行全量下载
- 日常更新使用增量更新功能
- 可以根据需要自定义股票池和参数